In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.models import Model
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
import numpy as np
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [3]:
!git clone https://github.com/rezaghoddoosian/Early-Drowsiness-Detection.git

Cloning into 'Early-Drowsiness-Detection'...
remote: Enumerating objects: 170, done.
remote: Total 170 (delta 0), reused 0 (delta 0), pack-reused 170
Receiving objects: 100% (170/170), 10.99 MiB | 5.61 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [4]:
import numpy as np
arr = np.load('./Early-Drowsiness-Detection/BlinksTest_30_Fold1.npy')

In [5]:
arr.shape 

(1269, 30, 4)

In [6]:
Blinks_train = np.array([]).reshape(0,30,4)
label_train = np.array([]).reshape(0,1)
for i in range(1,6) : 
    arr = np.load(f'./Early-Drowsiness-Detection/Blinks_30_Fold{i}.npy')
    arr_2 = np.load(f"./Early-Drowsiness-Detection/Labels_30_Fold{i}.npy")
    
    Blinks_train = np.vstack([Blinks_train,arr])
    label_train = np.append(label_train,arr_2)

In [7]:
Test_Blinks_train = np.array([]).reshape(0,30,4)
Test_label_train = np.array([]).reshape(0,1)
for i in range(1,6) : 
    arr = np.load(f'./Early-Drowsiness-Detection/BlinksTest_30_Fold{i}.npy')
    arr_2 = np.load(f"./Early-Drowsiness-Detection/LabelsTest_30_Fold{i}.npy")
    
    Test_Blinks_train = np.vstack([Blinks_train,arr])
    Test_label_train = np.append(label_train,arr_2)


In [8]:
Blinks_train.shape

(34592, 30, 4)

# Build a recurrent model to detect the state of te person 

In [9]:
data_dim = 4
timesteps = 30
num_classes = 11

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True,
               input_shape=(timesteps, data_dim)))) # returns a sequence of vectors of dimension 32
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  # returns a sequence of vectors of dimension 32
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(16)))  # return a single vector of dimension 32
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))


In [10]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience = 2,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.000001)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
model.fit(Blinks_train[:34500], label_train[:34500],
          batch_size=64, epochs=20,
          validation_data=(Test_Blinks_train, Test_label_train))

Epoch 1/20
540/540 [==============================] - 71s 108ms/step - loss: 0.9800 - accuracy: 0.5730 - val_loss: 0.5258 - val_accuracy: 0.7498
Epoch 2/20
540/540 [==============================] - 55s 102ms/step - loss: 0.4943 - accuracy: 0.7789 - val_loss: 0.3612 - val_accuracy: 0.8403
Epoch 3/20
540/540 [==============================] - 55s 102ms/step - loss: 0.3571 - accuracy: 0.8404 - val_loss: 0.2847 - val_accuracy: 0.8780
Epoch 4/20
540/540 [==============================] - 55s 102ms/step - loss: 0.2821 - accuracy: 0.8768 - val_loss: 0.1828 - val_accuracy: 0.9214
Epoch 5/20
540/540 [==============================] - 55s 102ms/step - loss: 0.2180 - accuracy: 0.9126 - val_loss: 0.1637 - val_accuracy: 0.9288
Epoch 6/20
540/540 [==============================] - 54s 101ms/step - loss: 0.1776 - accuracy: 0.9291 - val_loss: 0.1566 - val_accuracy: 0.9399
Epoch 7/20
540/540 [==============================] - 55s 101ms/step - loss: 0.1421 - accuracy: 0.9474 - val_loss: 0.1023 - val_ac

In [12]:
model.evaluate(Blinks_train[34500:], label_train[34500:])

3/3 [==============================] - 0s 16ms/step - loss: 0.1197 - accuracy: 0.9674


[0.11972814798355103, 0.967391312122345]

In [13]:
model.save('./lstm_blinks_model.h5')